In [1]:
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
def scrape_rest(main_link):
    r = {}
    try:
        for page in range(1,50000):
            link = main_link + '?page=' + str(page)
            soup = BeautifulSoup(requests.get(link).content,'html.parser')
            for rest in soup.find('div', class_ = '_129-b').find_all('div', class_ = '_3XX_A'):

                link_   = rest.find('a').get('href').split('/')[-1]                           # Restaurant Link
                id_     = rest.find('a').get('href').split('-')[-1]                           # Restaurant ID
                name_   = rest.find('div', class_ = 'nA6kb').text                             # Restaurant Name
                type_   = rest.find('div', class_ = '_1gURR').text                            # Type
                rating_ = rest.find('div', class_ = '_3Mn31').find_all('div')[0].text         # Rating
                price_  = rest.find('div', class_ = '_3Mn31').find_all('div')[-1].text        # Price

                r[id_] = {"name" : name_, "type" : type_, "rating" : rating_, "price" : price_ , "link" : link_}

    except:
        pass
    
    return r

In [3]:
fd = open('cities.json','r')
dct = json.loads(fd.read())
fd.close()

In [4]:
c = 0

for i in tqdm(dct.keys()):
    if (str(type(dct[i])) == "<class 'str'>"):
        link = dct[i]
        dct[i] = {'link' : dct[i], 'restaurants' : scrape_rest(link)}      
        
    else:

        temp = {}
        for j in dct[i].keys(): 
            temp[j] = {'link' : dct[i][j] ,'restaurants' : scrape_rest(dct[i][j])}
        dct[i] = temp
    
    c += 1

100%|███████████████████████████████████████| 623/623 [1:55:38<00:00, 11.14s/it]


In [6]:
fd = open('rest.json','w')
fd.write(json.dumps(dct))
fd.close()